In [39]:
import numpy as np
from scipy import ndimage
import os
import networkx as nx
from collections import defaultdict
import random

In [40]:
def find_connected_components(volume, structure):
    """Label connected components in a 3D binary volume."""
    labeled_volume, num_components = ndimage.label(volume, structure=structure)  # Label each connected component
    return labeled_volume, num_components

def voxel_graph(component, label_value):
    """Create a graph from a single connected component."""
    G = nx.Graph()
    dims = component.shape
    for i in range(dims[0]):
        for j in range(dims[1]):
            for k in range(dims[2]):
                if component[i, j, k] == label_value:
                    G.add_node((i, j, k))
                    #for di, dj, dk in [(1,0,0), (-1,0,0), (0,1,0), (0,-1,0), (0,0,1), (0,0,-1)]: ## not the right connectivity ?
                    for di in range(-1,2,1):
                        for dj in range(-1,2,1):
                            for dk in range(-1,2,1):
                                ni, nj, nk = i + di, j + dj, k + dk
                                if 0 <= ni < dims[0] and 0 <= nj < dims[1] and 0 <= nk < dims[2] and not (ni==i and nj==j and nk==k): # and not (np.linalg.norm((ni-i, nj-j, nk-k))>1.5) # use connectivity 26
                                    if component[ni, nj, nk] == label_value:
                                        G.add_edge((i, j, k), (ni, nj, nk), weight=np.linalg.norm((ni-i, nj-j, nk-k)))
    return G

In [41]:
def random_walk_all_points(graph, walk_length=20, alpha=0.5):
    """Simulates random walks from every point in the graph to measure accessibility."""
    visit_counts = defaultdict(int)
    
    for start_point in graph.nodes:
        current_point = start_point
        
        for _ in range(walk_length):
            visit_counts[current_point] += 1  # Count visit

            neighbors = list(graph.neighbors(current_point))
            if not neighbors:
                break  # Dead end

            # Move to a random neighbor
            current_point = random.choice(neighbors)
    
    # Normalize visit counts
    max_visits = max(visit_counts.values()) if visit_counts else 1
    for node in graph.nodes:
        #visit_counts[node] = visit_counts[node]*(1-alpha)/max_visits+alpha if node in visit_counts else alpha ## minimum on edges
        visit_counts[node] = (1-alpha)*(1-visit_counts[node]/max_visits) + alpha if node in visit_counts else 1 ## maximum on edges


    return visit_counts

# First test with Numpy

In [23]:
skeletons = np.load('/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/F.I.P./mask/Rskeleton.npy')
skel = skeletons[0]
volume = skel[:,:,:,0]!=0
structure = np.ones((3,3,3))
alpha = 0.5

In [ ]:
# Find connected components
labeled_volume, num_components = find_connected_components(volume, structure=structure)
print(f'Number of connected components : {num_components}')
print(np.unique(labeled_volume, return_counts=True)[1])

# random walks on each connected component
results = defaultdict(dict)
centers = {}
for label_value in range(1, num_components + 1):  # Labels start from 1
    G = voxel_graph(labeled_volume, label_value)
    visit_frequencies = random_walk_all_points(G, walk_length=100, alpha=alpha)
    for point, freq in list(visit_frequencies.items())[:10]:
        print(f"Point {point} -> Frequency {freq:.2f}")
    results = {**results, **visit_frequencies}

ccrdwalk_skel = np.zeros(skel.shape, dtype=float)
for point, freq in results.items():
    ccrdwalk_skel[point] = freq

Number of connected components : 18
[74957    23  1063     4     1     8     5     1   871     4    60     8
     1     1    17   183     6     4     3]
Point (3, 8, 26) -> Frequency 0.60
Point (4, 9, 25) -> Frequency 0.83
Point (4, 10, 25) -> Frequency 0.83
Point (5, 11, 24) -> Frequency 0.93
Point (4, 11, 24) -> Frequency 0.79
Point (5, 10, 25) -> Frequency 1.00
Point (6, 11, 24) -> Frequency 1.00
Point (7, 10, 25) -> Frequency 0.94
Point (8, 9, 24) -> Frequency 0.93
Point (8, 8, 25) -> Frequency 0.83
Point (4, 6, 18) -> Frequency 0.63
Point (5, 6, 18) -> Frequency 0.76
Point (5, 6, 17) -> Frequency 0.76
Point (5, 7, 16) -> Frequency 0.67
Point (6, 7, 16) -> Frequency 0.74
Point (6, 8, 16) -> Frequency 0.59
Point (7, 8, 15) -> Frequency 0.78
Point (8, 9, 15) -> Frequency 0.91
Point (8, 9, 16) -> Frequency 0.84
Point (9, 8, 16) -> Frequency 0.82
Point (4, 9, 21) -> Frequency 0.81
Point (5, 9, 20) -> Frequency 1.00
Point (4, 10, 19) -> Frequency 0.71
Point (5, 9, 21) -> Frequency 0.83


# With AIMS

In [42]:
import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt
from soma import aims

a = ana.Anatomist()

In [46]:
#skel_dir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/S.C.-sylv./mask/Lcrops'
skel_dir = '/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/F.I.P./mask/Rcrops'

alpha=0.5
walk_length=2000
nb_run = 1 ## to test stability of the random walk
nb_subjects = 16

subjects_list = os.listdir(skel_dir)
subjects_list = [elem for elem in subjects_list if elem[-1]!='f']
subjects_list = subjects_list[:nb_subjects]

volume_files = []
rdwalk_files = []


for subject in subjects_list:
    for _ in range(nb_run):
        skel_volume = aims.read(os.path.join(skel_dir, subject))
        skel_volume.np[skel_volume.np!=0]=1
        print(np.unique(skel_volume, return_counts=True))
        skel = skel_volume.np
        volume = skel[:,:,:,0]!=0
        structure = np.ones((3,3,3))

        # Find connected components
        labeled_volume, num_components = find_connected_components(volume, structure=structure)

        # random walks on each connected component
        results = defaultdict(dict)
        for label_value in range(1, num_components + 1):  # Labels start from 1
            G = voxel_graph(labeled_volume, label_value)
            visit_frequencies = random_walk_all_points(G, walk_length=walk_length, alpha=alpha)
            results = {**results, **visit_frequencies}

        ccrdwalk_skel = np.zeros(skel.shape, dtype=float)
        for point, freq in results.items():
            ccrdwalk_skel[point] = freq
        
        vol = ccrdwalk_skel.astype(np.float32)
        vol = aims.Volume(vol)

        volume_files.append(skel_volume)
        rdwalk_files.append(vol)

(array([0, 1], dtype=int16), array([74949,  2271]))
(array([0, 1], dtype=int16), array([75052,  2168]))
(array([0, 1], dtype=int16), array([75144,  2076]))
(array([0, 1], dtype=int16), array([75153,  2067]))
(array([0, 1], dtype=int16), array([75125,  2095]))
(array([0, 1], dtype=int16), array([74976,  2244]))
(array([0, 1], dtype=int16), array([74968,  2252]))
(array([0, 1], dtype=int16), array([75357,  1863]))
(array([0, 1], dtype=int16), array([74956,  2264]))
(array([0, 1], dtype=int16), array([74940,  2280]))
(array([0, 1], dtype=int16), array([75190,  2030]))
(array([0, 1], dtype=int16), array([75004,  2216]))
(array([0, 1], dtype=int16), array([74863,  2357]))
(array([0, 1], dtype=int16), array([75017,  2203]))
(array([0, 1], dtype=int16), array([75065,  2155]))
(array([0, 1], dtype=int16), array([75030,  2190]))


In [47]:
def build_gradient(pal):
    gw = ana.cpp.GradientWidget(None, 'gradientwidget', pal.header()['palette_gradients'])
    gw.setHasAlpha(True)
    nc = pal.shape[0]
    rgbp = gw.fillGradient(nc, True)
    rgb = rgbp.data()
    npal = pal.np['v']
    pb = np.frombuffer(rgb, dtype=np.uint8).reshape((nc, 4))
    npal[:, 0, 0, 0, :] = pb
    npal[:, 0, 0, 0, :3] = npal[:, 0, 0, 0, :3][:, ::-1]  # BGRA -> RGBA
    pal.update()

In [48]:
block = a.createWindowsBlock(4)
dic_windows = {}

for i, vol in enumerate(rdwalk_files):
    dic_windows[f'a_vol{i}'] = a.toAObject(vol)
    pal = a.createPalette('plasma')
    pal.header()['palette_gradients'] = f"0;0;0;0;0.05;0.4667;0.05;0.4667;0.1;0.5333;0.1;0.5333;0.15;0;0.15;0;0.2;0;0.2;0;0.25;0;0.25;0;0.3;0;0.3;0;0.35;0;0.35;0;0.4;0;0.4;0;0.45;0;0.45;0;0.5;0;0.5;0;0.55;0;0.55;0;0.6;0;0.6;0;0.65;0.7333;0.65;0.7333;0.7;0.9333;0.7;0.9333;0.75;1;0.75;1;0.8;1;0.8;1;0.85;1;0.85;1;0.9;0.8667;0.9;0.8667;0.95;0.8;0.95;0.8;1;0.8;1;0.8#0;0;0;0;0.05;0;0.05;0;0.1;0;0.1;0;0.15;0;0.15;0;0.2;0;0.2;0;0.25;0.4667;0.25;0.4667;0.3;0.6;0.3;0.6;0.35;0.6667;0.35;0.6667;0.4;0.6667;0.4;0.6667;0.45;0.6;0.45;0.6;0.5;0.7333;0.5;0.7333;0.55;0.8667;0.55;0.8667;0.6;1;0.6;1;0.65;1;0.65;1;0.7;0.9333;0.7;0.9333;0.75;0.8;0.75;0.8;0.8;0.6;0.8;0.6;0.85;0;0.85;0;0.9;0;0.9;0;0.95;0;0.95;0;1;0.8;1;0.8#0;0;0;0;0.05;0.5333;0.05;0.5333;0.1;0.6;0.1;0.6;0.15;0.6667;0.15;0.6667;0.2;0.8667;0.2;0.8667;0.25;0.8667;0.25;0.8667;0.3;0.8667;0.3;0.8667;0.35;0.6667;0.35;0.6667;0.4;0.5333;0.4;0.5333;0.45;0;0.45;0;0.5;0;0.5;0;0.55;0;0.55;0;0.6;0;0.6;0;0.65;0;0.65;0;0.7;0;0.7;0;0.75;0;0.75;0;0.8;0;0.8;0;0.85;0;0.85;0;0.9;0;0.9;0;0.95;0;0.95;0;1;0.8;1;0.8#0;0;0.2;0.977778;1;1"
    build_gradient(pal)
    dic_windows[f'a_vol{i}'].setPalette('plasma')
    dic_windows[f'rvol{i}'] = a.fusionObjects(objects=[dic_windows[f'a_vol{i}']], method='VolumeRenderingFusionMethod')
    dic_windows[f'rvol{i}'].releaseAppRef()
    dic_windows[f'wvr{i}'] = a.createWindow('3D', block=block)
    dic_windows[f'wvr{i}'].addObjects(dic_windows[f'rvol{i}'])

: 

array([0.        , 0.53472584, 0.54490864, ..., 0.9897163 , 0.9947055 ,
       1.        ], dtype=float32)